# **Sistema de Gestión de Servicios y Pasajes – ArgentinaTur**

In [1]:
from datetime import datetime, date
from abc import ABC, abstractmethod
from typing import List

# Medios de Pago
########################################################################################
# Medio de Pago (abstracta)
class MedioPago(ABC):
    @abstractmethod
    def realizar_pago(self, monto:float): # Metodo que varia segun la API
        pass

    @abstractmethod
    def ver_medio_pago(self):
        pass


# Tarjeta de Crédito
class TarjetaCredito(MedioPago):
    def __init__(self, numero:int, dni_titular:int, nombre:str, fecha_vencimiento:datetime.date):
        self.__numero = numero
        self.__dni_titular = dni_titular
        self.__nombre = nombre
        self.__fecha_vencimiento = fecha_vencimiento

    def realizar_pago(self, monto:float):
        print("Procesando pago con Tarjeta de credito ...")
        return True
    
    def ver_medio_pago(self):
        return "Tarjeta de Crédito"
    

# MercadoPago
class MercadoPago(MedioPago):
    def __init__(self, celular:int, email:str):
        self.__celular = celular
        self.__email = email

    def realizar_pago(self, monto:float):
        print("Procesando pago con Mercado Pago ...")
        return True
    
    def ver_medio_pago(self):
        return "Mercado Pago"
    

# Ualá
class Uala(MedioPago):
    def __init__(self, email:str, nombre_titular:str):
        self.__email = email
        self.__nombre_titular = nombre_titular

    def realizar_pago(self, monto:float):
        print("Procesando pago con Uala ...")
        return False  # Uala hoy no anda!!
    
    def ver_medio_pago(self):
        return "Ualá"
    
    
# Venta
########################################################################################
class Venta:
    def __init__(self, reserva:"Reserva", medio_pago : MedioPago, servicio:"Servicio"):
        self.__reserva = reserva
        self.__servicio = servicio
        self.__medio_pago = medio_pago
        self.__fecha = datetime.now()

    def ver_fecha(self):
        return self.__fecha
    
    def pagar(self):
        self.__reserva.confirmar_reserva()
        return self.__medio_pago.realizar_pago(self.__reserva.obtener_monto())
    
    def ver_monto(self):
        return self.__reserva.obtener_monto()

    def ver_medio_pago(self):
        return self.__medio_pago
    
    def ver_ubic_destino(self):
        return self.__servicio.ver_ubic_destino()
    
    def mostrar_info(self):
        print(f"  - Fecha: {self.__fecha}")
        print(f"    Servicio: {self.__servicio.ver_nombre()} ({self.__servicio.ver_loca_destino()})")
        print(f"    Medio de Pago: {self.__medio_pago.ver_medio_pago()}")
        print(f"    Monto: {self.__reserva.obtener_monto()}$")
    

# Pasajero
########################################################################################
class Pasajero:
    def __init__(self, nombre:str, email:str, dni:int):
        self.__nombre = nombre
        self.__email = email
        self.__dni = dni

    def obtener_nombre(self):
        return self.__nombre

# Reserva
########################################################################################
class Reserva:
    def __init__(self, fecha_hora:datetime, pasajero:Pasajero, asiento:"Asiento", nombre_serv:str, monto:float, nro:int):
        self.__nombre_serv = nombre_serv
        self.__fecha_hora = fecha_hora
        self.__pasajero = pasajero
        self.__asiento = asiento
        self.__confirmada = False
        self.__monto = monto
        self.__numero = nro

    def obtener_monto(self):
        return self.__monto

    def obtener_nro(self):
        return self.__numero

    def esta_confirmada(self):
        return self.__confirmada
    
    def ver_nombre_servicio(self):
        return self.__nombre_serv
    
    def confirmar_reserva(self):
        self.__confirmada = True

    def cancelar_reserva_si_no_esta_confirmada(self):
        if not (self.__confirmada):
            print(f"    Se cancelo la reserva de {self.__nombre_serv} número: {self.__numero}")
            self.__asiento.establecer_asiento_ocupado(False)

    def mostrar_reserva(self):
        print(f"Reserva del {self.__nombre_serv} con número: [{self.__numero}] realizada Correctamente")
        print(f"    Pasajero: {self.__pasajero.obtener_nombre()}, asiento: {self.__asiento.obtener_numero()}, {self.__nombre_serv}, fecha: {self.__fecha_hora}")

# unidad es abstracto, la declaro con distintos tipos de vehiculos
class Unidad(ABC):
    
    @abstractmethod
    def ver_asientos_disponibles(self):
        pass

    @abstractmethod
    def ocupar_asiento(self, numero_asiento):
        pass
      
    @abstractmethod
    def esta_ocupado_el_asiento(self, numero):
        pass
     
    @abstractmethod    
    def obtener_asiento(self, numero):
        pass
      

class Colectivo(Unidad):
    def __init__(self, patente, cantidad_asientos):
        self.__patente = patente
        self.__asientos = [Asiento(i) for i in range(cantidad_asientos)] # inicializar el map con todos los asientos (desocupados)
    
    def ver_asientos_disponibles(self):
        disponibles = []
        for asiento in self.__asientos:
            if not asiento.esta_ocupado():
                disponibles.append(asiento.obtener_numero())
        print("Asientos disponibles: ")
        print(" | ".join(map(str, disponibles)))
        print(" ")

    def ocupar_asiento(self, numero_asiento):
        self.__asientos[numero_asiento].establecer_asiento_ocupado(True)

    def esta_ocupado_el_asiento(self, numero):
        return self.__asientos[numero].esta_ocupado()
    
    def obtener_asiento(self, numero):
        return self.__asientos[numero]
    
# Asiento
########################################################################################
class Asiento:
    def __init__(self, numero:int):
        self.__numero = numero
        self.__ocupado = False # Se crean desocupados

    def establecer_asiento_ocupado(self,estado:bool):
        self.__ocupado = estado

    def esta_ocupado(self):
        return self.__ocupado
    
    def obtener_numero(self):
        return self.__numero

# Servicio
########################################################################################
class Servicio:
    # Inicializar parametros servicio
    def __init__(self, nombre:str, unidad:Unidad, fecha_partida:datetime, fecha_llegada:datetime, calidad:str, precio:float, itinerario:"Itinerario"):
        self.__nombre = nombre
        self.__unidad = unidad
        self.__fecha_partida = fecha_partida
        self.__fecha_llegada = fecha_llegada
        self.__calidad = calidad
        self.__precio = precio
        self.__itinerario = itinerario
        self.__disponible = True
        self.__reservas = {}

    # Funcion que cancela las reservas no confirmadas del servicio actual
    def cancelar_reservas_no_confirmada(self): 
        for reserva_actual in self.__reservas.values():
            reserva_actual.cancelar_reserva_si_no_esta_confirmada()
            if not (reserva_actual.esta_confirmada()):
                self.__reservas[reserva_actual.obtener_nro()] = None
    
    # Imprimir en Pantalla toda la información del servicio
    def mostrar_servicio(self): 
        if(self.__disponible):
            print(self.__nombre)
            print(f"Calidad: {self.__calidad}")
            print(f"Fecha de partida: {self.__fecha_partida}")
            print(f"fecha de llegada: {self.__fecha_llegada}")
            self.__itinerario.mostrar_itinerario()
            print(" ")
            self.mostrar_asientos_disponibles()

    def ver_nombre(self):
        return self.__nombre
    
    # Añade una reserva
    def aniadir_reserva(self, pasajero:Pasajero, numero_asiento:int, fecha_hora:datetime):
        # Comprueba si no esta ocupado el asiento y realiza la reserva
        if not (self.__unidad.esta_ocupado_el_asiento(numero_asiento)):
            self.__unidad.ocupar_asiento(numero_asiento)
            numero_reserva = self.obtener_numero_de_reserva()
            nueva_reserva = Reserva(fecha_hora, pasajero, self.__unidad.obtener_asiento(numero_asiento), self.__nombre, self.__precio, numero_reserva)
            self.__reservas[numero_reserva] = nueva_reserva
            nueva_reserva.mostrar_reserva()
            self.__unidad.ver_asientos_disponibles()
        else:
            print(f"Error. El asiento: '{numero_asiento}' ya esta ocupado.")
            print(" ")
            
    # Obtengo el numero de reserva en base a la cantidad existente
    def obtener_numero_de_reserva(self):
        return len(self.__reservas) 
    
    # Mostrar asientos disponibles
    def mostrar_asientos_disponibles(self):
        self.__unidad.ver_asientos_disponibles()

    def obtener_reserva(self, nro_reserva:int):
        return self.__reservas[nro_reserva]
    
    def obtener_nombre(self):
        return self.__nombre
    
    def ver_ubic_destino(self):
        return self.__itinerario.ver_ubic_destino()
    
#Ubicación, interfaz con posibilidad de extensión
class Ubicacion(ABC):
    @abstractmethod
    def obtener_info(self):
        pass
    @abstractmethod
    def ver_tipo_ubicacion(self):
        pass
    
class Ciudad(Ubicacion):
    def __init__(self, codigo, nombre, provincia):
        self.__codigo = codigo
        self.__nombre = nombre
        self.__provincia = provincia

    def obtener_info(self):
        return f"{self.__codigo} - {self.__nombre} / {self.__provincia}"
    
    def ver_tipo_ubicacion(self):
        return f"Ciudad"
    
class Barrio(Ubicacion):
    def __init__(self, nombre,ciudad):
        self.__nombre = nombre
        self.__ciudad = ciudad

    def obtener_info(self):
        return f"Barrio {self.__nombre} - {self.__ciudad}"
    
    def ver_tipo_ubicacion(self):
        return f"Barrio"
#---------------------------------------------------------------------- ----------------------------------------------------------------------

# Itinerario
########################################################################################
class Itinerario:
    def __init__(self, nombre:str, ubicaciones: list['Ubicacion']):
        self.__nombre = nombre
        self.__ubicaciones = ubicaciones

    def obtener_nombre(self):
        return self.__nombre
    
    def ver_ubic_destino(self):
        return self.__ubicaciones[-1].obtener_info()

    def mostrar_itinerario(self):
        print("Itinerario: ", self.__nombre)
        print(f"    {self.__ubicaciones[0].ver_tipo_ubicacion()} de origen:  {self.__ubicaciones[0].obtener_info()}")

        if len(self.__ubicaciones) > 2: # Si hay ubicaciones de parada las mostramos
            print("    Paradas: ")
            contador = 1
            for ubicacion in self.__ubicaciones[1:-1]:  # Slicing para iterar desde el segundo hasta el penúltimo elemento
                print(f"                [{contador}]:  ",ubicacion.obtener_info())
                contador += 1
        
        print(f"    {self.__ubicaciones[-1].ver_tipo_ubicacion()} de destino: {self.__ubicaciones[-1].obtener_info()}")


class ventasManager:
    def __init__(self):
        self.__ventas = []

    def agregar_venta(self, venta: Venta):
        self.__ventas.append(venta)

    def realizar_resumen(self, fecha_ini, fecha_fin):
        suma_monto = 0
        conteo_pagos = {}
        localidades_destino = {}

        for v in self.__ventas:
            if fecha_ini <= v.ver_fecha().date() <= fecha_fin:
                suma_monto += v.ver_monto()
                medio_pago = v.ver_medio_pago()
                if medio_pago not in conteo_pagos: 
                    conteo_pagos[medio_pago] = 0
                conteo_pagos[medio_pago] += 1
                destino = v.ver_ubic_destino()
                if destino not in localidades_destino:
                    localidades_destino[destino] = 0
                localidades_destino[destino] += 1
        respuesta = (f"\nEntre {fecha_ini} y {fecha_fin}:\n")
        respuesta += (f"\nMonto total recaudado: {suma_monto}$\n")
        respuesta += ("\nCantidad de pagos según medio de pago:\n")
        for medio_pago, cantidad in conteo_pagos.items():
            respuesta += (f"- {medio_pago.ver_medio_pago()}: {cantidad} \n")

        respuesta += ("\nUbicaciones con viajes comprados: \n")
        for destino, cantidad in localidades_destino.items():
            respuesta += (f"- {destino}: {cantidad} viajes \n")

        return respuesta
        


########################################################################################
# Sistema Argentur
########################################################################################
class Argentur:
    def __init__(self, sistema_activo:bool):
        self.__ventasmanager = ventasManager()
        self.__sistema_activo = sistema_activo
        self.__itinerarios = {} # Maps de Itinerarios, se accede según el nombre
        self.__servicios = {}   # Maps de Servicios disponibles
        
    # Itinerario
    ########################################################################################
    
    # Agregar Itinerarios al sistema
    def agregar_itinerario(self, itinerario:Itinerario):
        nombre = itinerario.obtener_nombre()
        if nombre in self.__itinerarios:
            print(f"Error el itinerario '{nombre}' ya existe.")
            print(" ")
        else:
            self.__itinerarios[nombre] = itinerario
   
    # Acceder a un itineario por su nombre
    def obtener_itinerario_por_nombre(self, nombre_it:Itinerario):
        return self.__itinerarios[nombre_it]

    # Servicios
    ########################################################################################
    
    # Agregar Servicios al sistema
    def agregar_servicio(self,nombre:str, unidad:Unidad, fecha_partida:datetime, fecha_llegada:datetime, calidad:str, precio:float, itinerario:Itinerario):
        servicio = Servicio(nombre, unidad, fecha_partida, fecha_llegada, calidad, precio, itinerario)
        if nombre in self.__servicios:
            print(f"Error el servicio '{nombre}' ya existe.")
            print(" ")
        else:
            self.__servicios[nombre] = servicio
    
    # Muestra los servicios disponibles
    def consultar_servicio_disponibles(self):
        for servicio in self.__servicios.values():
            servicio.mostrar_servicio()

    # Obtiene el servicio según el nombre
    def obtener_servicio_por_nombre(self, nombre: str):
        serv_aux = None
        for servicio in self.__servicios.values():
            if servicio.ver_nombre() == nombre:
                serv_aux = servicio
                break
        return serv_aux

    # Reservas
    ########################################################################################
    
    # Añade una reserva nueva
    def agregar_reserva(self, nombre_servicio:str, numero_asiento:int, pasajero:Pasajero):
        
        servicio_seleccionado = self.__servicios.get(nombre_servicio)

        if servicio_seleccionado is not None:
            fecha_hora = datetime.today()
            servicio_seleccionado.aniadir_reserva( pasajero, numero_asiento, fecha_hora)
        else:
            # Si el servicio no existe
            print("Error el servicio seleccionado no existe.")
            print(" ")

    # Obtiene una reserva existente, según el servicio y su número
    def obtener_reserva(self, nombre_servicio:str, nro_reserva:int):
        servicio_seleccionado = self.__servicios.get(nombre_servicio)
        if servicio_seleccionado is not None:
                reserva = servicio_seleccionado.obtener_reserva(nro_reserva)
        else:
            print("No existe ese número de reserva!")
            print(" ")
        return reserva

    # Cancela todas las reservas que nose pagaron
    def cancelar_reservas_no_pagadas(self):
        print(" ")
        print("Cancelando reservas no pagadas ...")
        for servicio in self.__servicios.values():
            servicio.cancelar_reservas_no_confirmada()
    # Ventas
    ########################################################################################

    # Se crean y guardan las ventas al momento de pagar la misma
    def pagar_reserva(self, reserva:Reserva, medio_pago:MedioPago):
        nombre_servicio = reserva.ver_nombre_servicio()
        serv = self.obtener_servicio_por_nombre(nombre_servicio)
        venta = Venta(reserva, medio_pago, serv)
        if (venta.pagar()):
            self.__ventasmanager.agregar_venta(venta)
            print("Pago aceptado!")
            print(" ")
        else:
            print("Pago rechazado!")
            print(" ")

    # Generar informe de ventas
    def mostrar_informe_de_ventas(self, fecha_ini:datetime.date, fecha_fin:datetime.date):
        print(f"{self.__ventasmanager.realizar_resumen(fecha_ini, fecha_fin)}")



    

## **Ejemplos de uso del Sistema Argentur**

In [2]:

Sistema_Argentur = Argentur(True)

calidad_economico = "Económico"
calidad_premium = "Premium"

# Crear Cuiudades
#                    Cod , Ciudad                   , Localidad
santoTome   = Ciudad(3016, "Santo Tomé"             , "Santa Fe")
santaFe     = Ciudad(3000, "Santa Fe"               , "Santa Fe")
parana      = Ciudad(3100, "Paraná"                 , "Entre Ríos")
rosario     = Ciudad(2000, "Rosario"                , "Santa Fe")
cordoba     = Ciudad(5000, "Córdoba"                , "Córdoba")
mendoza     = Ciudad(5500, "Mendoza"                , "Mendoza")
sanLuis     = Ciudad(5700, "San Luis"               , "San Luis")
neuquen     = Ciudad(8300, "Neuquén"                , "Neuquén")
bahiaBlanca = Ciudad(8000, "Bahía Blanca"           , "Buenos Aires")
marDelPlata = Ciudad(7600, "Mar del Plata"          , "Buenos Aires")
salta       = Ciudad(4400, "Salta"                  , "Salta")
resistencia = Ciudad(3500, "Resistencia"            , "Chaco")
posadas     = Ciudad(3300, "Posadas"                , "Misiones")
tucuman     = Ciudad(4000, "San Miguel de Tucumán"  , "Tucumán")
laPlata     = Ciudad(1900, "La Plata"               , "Buenos Aires")

#           Salida, Parada_1, Parada_2, Parada_n, Destino
ciudades1 = [santoTome, santaFe, parana]
ciudades2 = [parana, santaFe, santoTome]
ciudades3 = [rosario, santaFe, santoTome, parana, cordoba]
ciudades4 = [cordoba, rosario, parana, santoTome, santaFe]
ciudades5 = [mendoza, sanLuis, cordoba, rosario, santaFe, parana]

barrio1     = Barrio("Pepe", "Parana")
barrio2     = Barrio("Pipo", "Parana")
barrio3     = Barrio("Papo", "Parana")
barrio4     = Barrio("Santa Rosa", "Santa Fe")

barrios1    = [barrio1,barrio2,barrio3,barrio4]

# Creamos los Itinerarios de ejemplo, con un nombre y una lista de ciudades
itinerario1 = Itinerario("Viaje de Santo Tomé a Paraná", ciudades1)
itinerario2 = Itinerario("Viaje de Paraná a Santo Tomé", ciudades2)
itinerario3 = Itinerario("Tour Rosario a Córdoba", ciudades3)
itinerario4 = Itinerario("De Córdoba a Santa Fe pasando por Paraná", ciudades4)
itinerario5 = Itinerario("Gran Ruta Mendoza-Paraná", ciudades5)
itinerario6 = Itinerario("Recorrido por barrios", barrios1)

# Agregamos los Itinerarios al sistema
Sistema_Argentur.agregar_itinerario(itinerario1)
Sistema_Argentur.agregar_itinerario(itinerario2)
Sistema_Argentur.agregar_itinerario(itinerario3)
Sistema_Argentur.agregar_itinerario(itinerario4)
Sistema_Argentur.agregar_itinerario(itinerario5)
Sistema_Argentur.agregar_itinerario(itinerario6)

#Crear 9 colectivos
colectivo1 = Colectivo("AA123BB", 45)
colectivo2 = Colectivo("BB234CC", 50)
colectivo3 = Colectivo("CC345DD", 40)
colectivo4 = Colectivo("DD456EE", 48)
colectivo5 = Colectivo("EE567FF", 46)

# Crear 9 servicios
Sistema_Argentur.agregar_servicio("Servicio Santo Tomé - Paraná", colectivo1, datetime(2025, 5, 10, 8, 0), datetime(2025, 5, 10, 11, 30), calidad_economico, 15000, itinerario1)
# Intento de agregar el mismo servicio
Sistema_Argentur.agregar_servicio("Servicio Santo Tomé - Paraná", colectivo1, datetime(2025, 5, 10, 8, 0), datetime(2025, 5, 10, 11, 30), calidad_economico, 15000, itinerario1)
# Agrego los otros
Sistema_Argentur.agregar_servicio("Servicio Paraná - Santo Tomé", colectivo2, datetime(2025, 5, 11, 9, 0), datetime(2025, 5, 11, 12, 0), calidad_economico, 15000, itinerario2)
Sistema_Argentur.agregar_servicio("Servicio Rosario - Córdoba", colectivo3, datetime(2025, 5, 12, 7, 30), datetime(2025, 5, 12, 13, 45), calidad_premium, 32000, itinerario3)
Sistema_Argentur.agregar_servicio("Servicio Córdoba - Santa Fe", colectivo4, datetime(2025, 5, 13, 10, 0), datetime(2025, 5, 13, 17, 0), calidad_premium, 30000, itinerario4)
Sistema_Argentur.agregar_servicio("Recorrido por barrios", colectivo5, datetime(2025, 5, 14, 6, 0), datetime(2025, 5, 14, 20, 0), calidad_premium, 60000, itinerario6)

# Mostrar Todos los servicios disponibles
Sistema_Argentur.consultar_servicio_disponibles()

# Creamos pasajeros de ejemplo
pasajero1 = Pasajero("Tomas", "tomi@gmail.com", 42704142)
pasajero2 = Pasajero("Maria", "maria@hotmail.com", 34987654)
pasajero3 = Pasajero("Juan", "juan123@yahoo.com", 30874561)
pasajero4 = Pasajero("Lucia", "lucia@outlook.com", 31746528)
pasajero5 = Pasajero("Pepe", "pepe@outlook.com", 72451282)

# Agregamos reservas
Sistema_Argentur.agregar_reserva("Servicio Santo Tomé - Paraná", 3, pasajero1)
Sistema_Argentur.agregar_reserva("Servicio Paraná - Santo Tomé", 10, pasajero2)
Sistema_Argentur.agregar_reserva("Servicio Paraná - Santo Tomé", 10, pasajero3) # Ejemplo de reserva del mismo asiento
Sistema_Argentur.agregar_reserva("Servicio Paraná - Santo Tomé", 9, pasajero4)
Sistema_Argentur.agregar_reserva("Servicio que no existe", 2, pasajero4)        # Ejemplo de servicio que no existe
Sistema_Argentur.agregar_reserva("Servicio Córdoba - Santa Fe", 7, pasajero5)
Sistema_Argentur.agregar_reserva("Recorrido por barrios", 7, pasajero5)

# Busco la reserva para luego pagarlas
reserva1 = Sistema_Argentur.obtener_reserva("Servicio Santo Tomé - Paraná", 0)
reserva2 = Sistema_Argentur.obtener_reserva("Servicio Paraná - Santo Tomé", 0)
reserva3 = Sistema_Argentur.obtener_reserva("Servicio Paraná - Santo Tomé", 1)
reserva4 = Sistema_Argentur.obtener_reserva("Servicio Córdoba - Santa Fe", 0) # Esta reserva nunca se paga
reserva5 = Sistema_Argentur.obtener_reserva("Servicio Córdoba - Santa Fe", 0)
reserva6 = Sistema_Argentur.obtener_reserva("Recorrido por barrios", 0)

# Medios de pago
medioPago1 = MercadoPago(3424499685, "tomi@gmail.com")
medioPago2 = Uala("maria@hotmail.com", "Maria")     
medioPago3 = TarjetaCredito(1234456789101112, 30874561,"Juan", datetime(2029, 5, 18, 6, 0))

# Pagar las reservas
Sistema_Argentur.pagar_reserva(reserva1,medioPago1)
Sistema_Argentur.pagar_reserva(reserva2,medioPago2) # Uala no anda hoy!
Sistema_Argentur.pagar_reserva(reserva3,medioPago3)
Sistema_Argentur.pagar_reserva(reserva4,medioPago1)
Sistema_Argentur.pagar_reserva(reserva5,medioPago1)
Sistema_Argentur.pagar_reserva(reserva6,medioPago1)


fecha_inicio = date(2020, 5, 10)
fecha_fin = date(2026, 1, 1)

# Mostrar Informe
Sistema_Argentur.mostrar_informe_de_ventas(fecha_inicio,fecha_fin)

# Cancelar reservas no pagadas
Sistema_Argentur.cancelar_reservas_no_pagadas()

Error el servicio 'Servicio Santo Tomé - Paraná' ya existe.
 
Servicio Santo Tomé - Paraná
Calidad: Económico
Fecha de partida: 2025-05-10 08:00:00
fecha de llegada: 2025-05-10 11:30:00
Itinerario:  Viaje de Santo Tomé a Paraná
    Ciudad de origen:  3016 - Santo Tomé / Santa Fe
    Paradas: 
                [1]:   3000 - Santa Fe / Santa Fe
    Ciudad de destino: 3100 - Paraná / Entre Ríos
 
Asientos disponibles: 
0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 | 24 | 25 | 26 | 27 | 28 | 29 | 30 | 31 | 32 | 33 | 34 | 35 | 36 | 37 | 38 | 39 | 40 | 41 | 42 | 43 | 44
 
Servicio Paraná - Santo Tomé
Calidad: Económico
Fecha de partida: 2025-05-11 09:00:00
fecha de llegada: 2025-05-11 12:00:00
Itinerario:  Viaje de Paraná a Santo Tomé
    Ciudad de origen:  3100 - Paraná / Entre Ríos
    Paradas: 
                [1]:   3000 - Santa Fe / Santa Fe
    Ciudad de destino: 3016 - Santo Tomé / Santa Fe
 
Asientos disponibles: 
0 | 1 | 2